###### Import Dependencies & Chrome Driver

In [1]:
# import dependencies
from bs4 import BeautifulSoup as soup
import pandas as pd
from pprint import pprint
import pymongo
from splinter import Browser
import time

In [2]:
# choose path to driver (Chrome v91)
executable_path = {'executable_path' : 'chromedriver'}
browser = Browser(
    'chrome', 
    **executable_path, 
    headless = False
)

# delay to allow time for page to load
time.sleep(5)

###### Scraping Web Text

In [3]:
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client.mars_db
collection = db.mars_data

In [4]:
# navigate browser to url
browser.visit('https://mars.nasa.gov/news/')

# delay to allow time for page to load
time.sleep(5)

# create beautifulsoup object
article_soup = soup(browser.html, 'html.parser')

In [5]:
# narrow HTML elements by 'article'
article_result = article_soup.find('article')

# retrieve most recent article date, title, paragraph
news_date = article_result.find('div', class_ = 'list_date').text.strip()
news_title = article_result.find('div', class_ = 'content_title').text.strip()
news_p = article_result.find('div', class_ = 'article_teaser_body').text.strip()

In [6]:
# create dictionary for mongodb
recent_news = {
    'date' : news_date,
    'title' : news_title,
    'teaser' : news_p
}

###### Scraping Web Image

In [7]:
# navigate browser to url, click full-res image
browser.visit('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html')
browser.links.find_by_partial_text('FULL IMAGE').click()

# create beautifulsoup object
image_soup = soup(browser.html, 'html.parser')

# delay to allow time for page to load
time.sleep(5)

In [8]:
# extract image source url
image_result = image_soup.find(class_ = 'fancybox-image')['src']

In [9]:
# append urls
featured_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + image_result

In [10]:
# create dictionary for mongodb
featured_image = {
    'featured_image_url' : featured_image_url
}

###### Converting HTML table to DataFrame, convert back to HTML

In [11]:
# navigate browser to url
browser.visit('https://space-facts.com/mars/')

# create beautifulsoup object
fact_soup = soup(browser.html, 'html.parser')

# delay to allow time for page to load
time.sleep(5)

In [12]:
# convert html table to dataframe
fact_df = pd.read_html('https://space-facts.com/mars/')[0]

In [13]:
# convert dataframe to html
fact_html = fact_df.to_html(index = False, header = False)

In [14]:
# create dictionary for mongodb
facts = {
    'html_table' : fact_html
}

###### Mars Hemispheres

In [15]:
# navigate browser to url
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

# create beautifulsoup object
hemi_soup = soup(browser.html, 'html.parser')

# delay to allow time for page to load
time.sleep(5)

In [16]:
# grab hemisphere HTML elements
hemisphere_content = hemi_soup.find_all('div', class_ = 'item')

In [17]:
# create list for hemisphere info
hemispheres = {}

# retrieve hemisphere info
for hemisphere in hemisphere_content:
    # assign image titles for hemispheres
    hemi_name = hemisphere.find('div', class_='description').h3.text
    # delay to allow time for page to load
    time.sleep(1)
    browser.find_by_text(hemi_name).click()
    hemi_soup = soup(browser.html, 'html.parser')
    # find, append img href
    download = hemi_soup.find('div', class_ = "downloads")
    src = download.find('a')
    if src.text == 'Sample':
        hemi_url = src['href']
    hemispheres[hemi_name] = hemi_url
    time.sleep(1)
    browser.back()

In [18]:
# close browser
browser.quit()

In [19]:
# clear collection
collection.delete_many({})

# insert collection to mongodb
collection.insert_many([recent_news, featured_image, facts, hemispheres])

In [20]:
# query all documents from collection
for document in collection.find():
    # print each document
    pprint(document)

{'_id': ObjectId('60bd7678125006e376886f03'),
 'date': 'June  3, 2021',
 'teaser': 'The spacecraft successfully cleared some dust off its solar '
           'panels, helping to raise its energy and delay when it will need to '
           'switch off its science instruments.',
 'title': "NASA's InSight Mars Lander Gets a Power Boost"}
{'_id': ObjectId('60bd7678125006e376886f04'),
 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg'}
{'_id': ObjectId('60bd7678125006e376886f05'),
 'html_table': '<table border="1" class="dataframe">\n'
               '  <tbody>\n'
               '    <tr>\n'
               '      <td>Equatorial Diameter:</td>\n'
               '      <td>6,792 km</td>\n'
               '    </tr>\n'
               '    <tr>\n'
               '      <td>Polar Diameter:</td>\n'
               '      <td>6,752 km</td>\n'
               '    </tr>\n'
               '    <tr>\n'
               '      <td>Mass:</td>\n'
       